In [1]:
import pandas as pd
import numpy as np
# fix random seed for reproducibility
np.random.seed(7)

In [2]:
cleaned = pd.read_csv('datasets/cleaned_Izends_Data_Thru201712_ver5.csv',index_col=0)

/home/ambuj/izenda_fullcycle/environment/env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cleaned_loss_des = pd.read_csv('hashed_cleaned_loss_desc_from_5_ipynb.csv',index_col=0)
# changing dtype of hashed_loss_desc_padded column
cleaned_loss_des['hashed_loss_desc_padded'] = cleaned_loss_des['hashed_loss_desc_padded'].apply(lambda x: np.fromstring(x[1:-1],dtype='int32',sep=' '))

In [4]:
cleaned_combined = pd.merge(cleaned,cleaned_loss_des[['EstimateTotal_bins','hashed_loss_desc_padded']],left_index=True,right_index=True)

In [5]:
cleaned_combined.shape

(766408, 28)

In [6]:
cleaned_combined.columns

Index(['DisplayName', 'DivisionName', 'City', 'Zip', 'State', 'LossYearMo',
       'StartedFlag', 'CommOrRes', 'NoteCount', 'PhotoCount', 'JobCount',
       'ClaimCount', 'EstimateTotal', 'PolicyHolderType', 'LossDescription',
       'Estimate_NetClaim', 'Estimate_MaterialSaleTax', 'Estimate_OverHead',
       'Estimate_Profit', 'Estimate_PctOverhead', 'Estimate_PctProfit',
       'Estimate_Deductible', 'Estimate_BaseSvcCharge',
       'CleanAddressFranchisorID', 'CleanAddressContactID', 'LossMo',
       'EstimateTotal_bins', 'hashed_loss_desc_padded'],
      dtype='object')

# Defining X and Y

In [7]:
X1 = cleaned_combined[['DisplayName', 'DivisionName','LossMo','CommOrRes','PolicyHolderType','NoteCount','PhotoCount','JobCount','ClaimCount','LossYearMo']]
X2 = cleaned_combined['hashed_loss_desc_padded']
y = cleaned_combined['EstimateTotal_bins']

# Getting Dummies

In [8]:
X1_dummies = pd.get_dummies(X1,sparse=True)

In [9]:
all_data = pd.merge(X1_dummies,pd.DataFrame(X2),left_index=True,right_index=True)
all_data['EstimateTotal_bins'] = y

# Splitting data

In [10]:
test2017=all_data[all_data["LossYearMo"].apply(lambda all_data:all_data>=201701)]
learning=all_data[all_data["LossYearMo"].apply(lambda all_data:all_data<201701)]

In [11]:
learning_x = learning[learning.columns.difference(['LossYearMo','EstimateTotal_bins'])]
learning_y = learning['EstimateTotal_bins']
learning_y_dummies = pd.get_dummies(learning_y,sparse=True)

In [12]:
test2017_x1 = test2017[test2017.columns.difference(['LossYearMo','EstimateTotal_bins','hashed_loss_desc_padded'])]
test2017_x2 = test2017['hashed_loss_desc_padded']
test2017_y = test2017['EstimateTotal_bins']
test2017_y_dummies = pd.get_dummies(test2017_y,sparse=True)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(learning_x, learning_y_dummies, test_size=0.33, random_state=42)

In [14]:
X1_train = X_train[X_train.columns.difference(['hashed_loss_desc_padded'])]
X2_train = X_train['hashed_loss_desc_padded']

In [15]:
X1_val = X_val[X_val.columns.difference(['hashed_loss_desc_padded'])]
X2_val = X_val['hashed_loss_desc_padded']

# Building Custom Keras Combined Model using Functional API

In [16]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, History 
from keras.layers.merge import concatenate

Using TensorFlow backend.


In [17]:
# first input
input1 = Input(shape=(X1_train.shape[1],),name='input1')
hidden1 = Dense(512, activation='relu')(input1)
dropout1 = Dropout(0.25)(hidden1)
hidden2 = Dense(512, activation='relu')(dropout1)
dropout2 = Dropout(0.25)(hidden2)

In [18]:
# second input
input2 = Input(shape=(14,),name='input2')
embedding = Embedding(11024,100,input_length=14)(input2)
lstm = LSTM(15)(embedding)

In [19]:
# merge layers
merge = concatenate([dropout2,lstm])

In [20]:
# output layer
output = Dense(y_train.shape[1], activation='softmax',name='output')(merge)

In [21]:
# Model
model = Model(inputs=[input1,input2], outputs=output)

In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             (None, 1064)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 512)          545280      input1[0][0]                     
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 512)          0           dense_1[0][0]                    
__________________________________________________________________________________________________
input2 (InputLayer)             (None, 14)           0                                            
__________________________________________________________________________________________________
dense_2 (D

In [27]:
plot_model(model,to_file='combined_model.png')

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
early_stop = EarlyStopping(monitor='val_loss', patience=50,restore_best_weights=True)

In [30]:
model.fit({'input1':X1_train,'input2':np.stack(X2_train, axis=0)},
          {'output':y_train.values},
          validation_data=({'input1':X1_val,'input2':np.stack(X2_val, axis=0)},{'output':y_val.values}),
         callbacks=[early_stop],epochs=500, batch_size=128,verbose=1)

Train on 335176 samples, validate on 165087 samples
Epoch 1/500
335176/335176 [==============================] - 81s 240us/step - loss: 2.0813 - acc: 0.2117 - val_loss: 2.0188 - val_acc: 0.2310
Epoch 2/500
335176/335176 [==============================] - 78s 234us/step - loss: 2.0060 - acc: 0.2374 - val_loss: 2.0059 - val_acc: 0.2378
Epoch 3/500
335176/335176 [==============================] - 78s 233us/step - loss: 1.9792 - acc: 0.2491 - val_loss: 2.0025 - val_acc: 0.2384
Epoch 4/500
335176/335176 [==============================] - 78s 234us/step - loss: 1.9532 - acc: 0.2614 - val_loss: 2.0050 - val_acc: 0.2375
Epoch 5/500
335176/335176 [==============================] - 78s 234us/step - loss: 1.9274 - acc: 0.2734 - val_loss: 2.0209 - val_acc: 0.2358
Epoch 6/500
335176/335176 [==============================] - 78s 234us/step - loss: 1.9012 - acc: 0.2866 - val_loss: 2.0333 - val_acc: 0.2354
Epoch 7/500
335176/335176 [==============================] - 78s 234us/step - loss: 1.8771 - acc

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



335176/335176 [==============================] - 78s 234us/step - loss: 1.6494 - acc: 0.3878 - val_loss: 2.3395 - val_acc: 0.2230
Epoch 23/500
335176/335176 [==============================] - 78s 234us/step - loss: 1.6413 - acc: 0.3903 - val_loss: 2.3635 - val_acc: 0.2216
Epoch 24/500
335176/335176 [==============================] - 78s 234us/step - loss: 1.6329 - acc: 0.3943 - val_loss: 2.3753 - val_acc: 0.2240
Epoch 25/500
335176/335176 [==============================] - 78s 234us/step - loss: 1.6245 - acc: 0.3970 - val_loss: 2.3851 - val_acc: 0.2221
Epoch 26/500
335176/335176 [==============================] - 79s 234us/step - loss: 1.6183 - acc: 0.3989 - val_loss: 2.4026 - val_acc: 0.2217
Epoch 27/500
335176/335176 [==============================] - 78s 234us/step - loss: 1.6103 - acc: 0.4028 - val_loss: 2.4246 - val_acc: 0.2214
Epoch 28/500
335176/335176 [==============================] - 78s 234us/step - loss: 1.6042 - acc: 0.4045 - val_loss: 2.4350 - val_acc: 0.2202
Epoch 29/500

# Calculating Test Accuracy

In [34]:
test2017_preds_proba = model.predict({'input1':test2017_x1,'input2':np.stack(test2017_x2, axis=0)})

In [40]:
test2017_preds = np.argmax(test2017_preds_proba,axis=1)

In [41]:
mapping_dict = {0:'a',1:'b',2:'c',3:'d',4:'e',5:'f',6:'g',7:'h',8:'i',9:'j'}

In [42]:
test2017_preds_names = pd.Series(test2017_preds).map(mapping_dict)

In [44]:
from sklearn.metrics import accuracy_score

In [45]:
accuracy_score(test2017_y,test2017_preds_names)

0.23186984538503447